In [0]:
import keras
from keras import models
from keras import layers
from os.path import isfile
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Bidirectional, LSTM, Dropout, Activation, GRU
from keras.layers import Conv2D, concatenate, MaxPooling2D, Flatten, Embedding, Lambda

from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import backend as K
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop
from keras import regularizers


def conv_recurrent_model_build(model_input):
    print('Building model...')
    layer = model_input
    
    ### Convolutional blocks
    conv_1 = Conv2D(filters = nb_filters1, 
                    kernel_size = ksize, 
                    strides=1,
                    padding= 'valid', 
                    activation='relu', 
                    name='conv_1')(layer)
    
    pool_1 = MaxPooling2D(pool_size_1)(conv_1)

    conv_2 = Conv2D(filters = nb_filters2, 
                    kernel_size = ksize, 
                    strides=1,
                    padding= 'valid', 
                    activation='relu', 
                    name='conv_2')(pool_1)
    
    pool_2 = MaxPooling2D(pool_size_1)(conv_2)

    conv_3 = Conv2D(filters = nb_filters3, 
                    kernel_size = ksize, 
                    strides=1,
                    padding= 'valid', 
                    activation='relu', 
                    name='conv_3')(pool_2)

    pool_3 = MaxPooling2D(pool_size_2)(conv_3)

    flatten1 = Flatten()(pool_3)
    ### Recurrent Block
    
    # Pooling layer
    pool_lstm1 = MaxPooling2D(pool_size_3, 
                              name = 'pool_lstm')(layer)
    
    # Embedding layer
    squeezed = Lambda(lambda x: K.squeeze(x, axis= -1))(pool_lstm1)

    # Bidirectional GRU
    # default merge mode is concat
    lstm = Bidirectional(GRU(lstm_count))(squeezed)  
    
    # Concat Output
    concat = concatenate([flatten1, lstm], 
                         axis=-1, 
                         name ='concat')
    
    ## Softmax Output
    output = Dense(num_classes, 
                   activation = 'softmax', 
                   name='preds')(concat)
    
    model_output = output
    model = Model(model_input, model_output)
    
    opt = RMSprop(lr=0.0005)  # Optimizer
    model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])
    
    print(model.summary())
    return model
  
def train_model(x_train, y_train, x_val, y_val):
    
    n_frequency = 43
    n_frames = 256
    
    x_train = np.expand_dims(x_train, axis = -1)
    x_val = np.expand_dims(x_val, axis = -1)
  
    weight = np.ones((y_train.shape[0]))
    prog_len = len(y_train[y_train[:,0]==1])
    totallen = y_train.shape[0]
    weight[y_train[:,0] == 0] = prog_len/totallen
    weight[y_train[:,0] == 1] = 1 - prog_len/totallen
    sample_weight = weight
    
    input_shape = (n_frames, n_frequency, 1)
    model_input = Input(input_shape, name='input')
    
    model = conv_recurrent_model_build(model_input)
    
    checkpoint_callback = ModelCheckpoint('/path/weights.best.h5', 
                                          monitor='val_acc', 
                                          verbose=1,
                                          save_best_only=True, 
                                          mode='max')
    
    reducelr_callback = ReduceLROnPlateau(monitor='val_acc', 
                                          factor=0.5, 
                                          patience=10, 
                                          min_delta=0.01,
                                          verbose=1)
    
    callbacks_list = [checkpoint_callback, reducelr_callback]

    # Fit the model and get training history.
    print('Training...')
    history = model.fit(x_train, 
                        y_train, 
                        batch_size=BATCH_SIZE, 
                        epochs=EPOCH_COUNT,
                        sample_weight = sample_weight,
                        validation_data=(x_val, y_val), 
                        verbose=1, 
                        callbacks=callbacks_list)

    return model, history
  